# Statistical Language Models

This notebook provides an overview and examples of three types of statistical language models: n-grams, Hidden Markov Models (HMMs), and Maximum Likelihood Estimation (MLE).


In [1]:
with open('Alice.txt', 'r') as file:
    corpus = file.readlines()#.split('.')
A_corpus = [x.strip() for x in corpus if x != '\n']

## N-grams

An n-gram is a contiguous sequence of n items from a given sample of text or speech. The items can be phonemes, syllables, letters, words, or base pairs according to the application. N-grams are used in various areas of statistical natural language processing and genetic sequence analysis.


In [4]:
import random
from collections import defaultdict, Counter

class NGramLanguageModel:
    def __init__(self):
        # Initialize dictionaries to store bigram and trigram counts
        self.bigrams = defaultdict(Counter)
        self.trigrams = defaultdict(Counter)
        self.starting_bigrams = Counter()

    def train(self, corpus):
        """
        Train the language model using the provided corpus.
        The corpus should be a list of sentences.

        :param corpus: List of sentences (strings).
        """
        for sentence in corpus:
            # Tokenize the sentence with special tokens <start> and <end>
            tokens = ['<start>'] + sentence.split() + ['<end>']
            for i in range(len(tokens) - 2):
                # Count bigrams and trigrams
                self.bigrams[tokens[i]][tokens[i + 1]] += 1
                self.trigrams[(tokens[i], tokens[i + 1])][tokens[i + 2]] += 1
                # Track starting bigrams for sentence generation
                if i == 0:
                    self.starting_bigrams[tokens[i]] += 1

    def predict_next_word(self, word_sequence):
        """
        Predict the next word based on the given word sequence using the trained model.
        It first tries using the trigram model and then falls back to the bigram model.

        :param word_sequence: A string of word sequence.
        :return: The predicted next word, or None if no prediction is available.
        """
        tokens = word_sequence.split()
        # Check for trigram match
        if len(tokens) >= 2 and (tokens[-2], tokens[-1]) in self.trigrams:
            return self.trigrams[(tokens[-2], tokens[-1])].most_common(1)[0][0]
        # Fallback to bigram match
        elif tokens[-1] in self.bigrams:
            return self.bigrams[tokens[-1]].most_common(1)[0][0]
        else:
            # Return None if no match is found
            return None

# Example usage
corpus = [
    "the cat sat on the mat",
    "the dog sat on the rug",
    "the dog played with the cat",
    "the cat and the dog are friends"
]

model = NGramLanguageModel()
model.train(corpus)

# Predict the next word for bigram sequence
bigram_sequence = "the"
next_word_bigram = model.predict_next_word(bigram_sequence)
print(f"Next word after '{bigram_sequence}': {next_word_bigram}")

# Predict the next word for trigram sequence
trigram_sequence = "the dog"
next_word_trigram = model.predict_next_word(trigram_sequence)
print(f"Next word after '{trigram_sequence}': {next_word_trigram}")


Next word after 'the': cat
Next word after 'the dog': sat


In [5]:
model = NGramLanguageModel()
model.train(A_corpus)

# Predict the next word for bigram sequence
bigram_sequence = "Alice"
next_word_bigram = model.predict_next_word(bigram_sequence)
print(f"Next word after '{bigram_sequence}': {next_word_bigram}")

Next word after 'Alice': was


In [7]:
model = NGramLanguageModel()
model.train(A_corpus[200:1000])

res = []
word = 'Alice'
res.append(word)
for i in range(50):
    word = model.predict_next_word(word)
    res.append(word)
print(' '.join(res))

Alice was a little pattering of the White Rabbit blew three gardeners at the White Rabbit blew three gardeners at the White Rabbit blew three gardeners at the White Rabbit blew three gardeners at the White Rabbit blew three gardeners at the White Rabbit blew three gardeners at the White Rabbit


# Reasons for Repetitive Output in N-Gram Language Models

When using basic n-gram models, particularly bigrams, it's common to encounter outputs that get stuck in repetitive loops. Several factors contribute to this behavior:

## Strong Affinity Between Words

If the training corpus contains frequently repeating phrases, the model might learn a strong association between certain word pairs. For instance, if "the Queen said" occurs often, the model learns that "said" is likely to follow "Queen." This can cause a feedback loop where the model continually predicts the same sequence.

## Limited Context

Bigram models consider only the previous word to predict the next one. This limited context can lead to loops if a word pair like "Queen said" is commonly followed by "the," and "the" is commonly followed by "Queen" again.

## Sparse Data

Training on a small subset of the corpus may not expose the model to enough variety in word sequences, leading to overfitting on certain patterns. The model's output then reflects these overrepresented patterns.

## Lack of Smoothing

Without smoothing, n-gram models assign zero probability to unseen n-grams. This can lead to repetitive predictions since the model tends to choose the highest probability next word it has encountered during training.

## End-of-Sequence Handling

Simple n-gram models may not have a clear indication of when to end a sentence, leading to run-on sentences or repetitive loops, as they lack an understanding of natural sentence length variation.

## Solutions

To mitigate the repetition in generated text:

- Move to higher-order n-grams (trigrams, 4-grams, etc.) to provide more context for predictions.
- Implement smoothing techniques to better handle unseen word pairs.
- Use a larger and more diverse training corpus to provide a broader learning base for the model.
- Introduce mechanisms to detect and prevent loops in generated text, such as setting a maximum sentence length.


## Hidden Markov Models (HMMs)

A Hidden Markov Model is a statistical Markov model in which the system being modeled is assumed to be a Markov process with unobserved (hidden) states. HMMs can be considered as the simplest dynamic Bayesian networks. In a regular Markov model, the state is directly visible to the observer, and therefore the state transition probabilities are the only parameters, while in a hidden Markov model, the state is not directly visible, but the output, dependent on the state, is visible.


In [8]:
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder

class HMMWordGenerator:
    def __init__(self, n_states=3):
        """
        Initialize the HMMWordGenerator with a specified number of states.

        :param n_states: Number of hidden states in the HMM.
        """
        self.model = hmm.MultinomialHMM(n_components=n_states, random_state=42)
        self.label_encoder = LabelEncoder()

    def fit(self, sentences):
        """
        Train the HMM model on a given corpus.

        :param sentences: A list of sentences for training the model.
        """
        # Extract unique words from the sentences
        unique_words = set(word for sentence in sentences for word in sentence.split())
        # Fit the label encoder with the unique words
        self.label_encoder.fit(list(unique_words))

        # Transform sentences to sequences of numerical labels
        encoded_sequences = [self.label_encoder.transform(sentence.split()) for sentence in sentences]
        # Calculate the lengths of each sentence
        lengths = [len(seq) for seq in encoded_sequences]
        # Combine all sequences into a single NumPy array for HMM training
        X = np.concatenate(encoded_sequences).reshape(-1, 1)

        # Fit the HMM model to the data
        self.model.fit(X, lengths=lengths)

    def sample(self, start_word, max_length=10):
        """
        Generate a sequence of words starting with the given word.

        :param start_word: The word to start the sequence.
        :param max_length: Maximum length of the generated sequence.
        :return: A string representing the generated sequence.
        """
        # Encode the starting word
        start_word_encoded = self.label_encoder.transform([start_word])[0]
        word_sequence = [start_word_encoded]

        # Generate the sequence
        for _ in range(max_length - 1):
            current_state = word_sequence[-1]
            # Check if the current state is within the bounds of the transition matrix
            if current_state < self.model.transmat_.shape[0]:
                # Choose the next word based on the transition probabilities
                next_word_prob = self.model.transmat_[current_state]
                next_word_encoded = np.random.choice(len(next_word_prob), p=next_word_prob)
                word_sequence.append(next_word_encoded)
            else:
                break

        # Decode the sequence back to words
        return ' '.join(self.label_encoder.inverse_transform(word_sequence))

# Example usage
sentences = [
    "the cat sat on the mat",
    "the dog sat on the rug",
    "the dog played with the cat",
    "the cat and the dog are friends"
]

# Initialize and train the HMM model
hmm_model = HMMWordGenerator(n_states=5)
hmm_model.fit(sentences)

# Generate a sequence starting with 'the'
print(hmm_model.sample('cat'))


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


cat are friends and are friends and are friends and


## Maximum Likelihood Estimation (MLE)

Maximum Likelihood Estimation (MLE) is a method of estimating the parameters of a statistical model. In the context of language models, MLE is used to estimate the probabilities of different words (or sequences of words) occurring. The idea is to choose the parameters of a model in such a way that the likelihood of the observed data is maximized.


In [23]:
from collections import defaultdict, Counter

class MLELanguageModel:
    def __init__(self):
        # Dictionary to store bigram counts
        self.bigram_counts = defaultdict(Counter)
        # Dictionary to store unigram counts
        self.unigram_counts = Counter()

    def train(self, corpus):
        """
        Train the language model on a given corpus.
        
        :param corpus: A list of sentences (strings) for training the model.
        """
        for sentence in corpus:
            tokens = sentence.split()
            # Update unigram and bigram counts
            for i in range(len(tokens)):
                self.unigram_counts[tokens[i]] += 1
                if i < len(tokens) - 1:
                    self.bigram_counts[tokens[i]][tokens[i + 1]] += 1

    def predict_next_word(self, word):
        """
        Predict the next word given a word using the trained model.
        
        :param word: The input word to base the prediction on.
        :return: The predicted next word.
        """
        if word not in self.bigram_counts:
            return None
        # Predict the next word based on maximum likelihood estimation
        return self.bigram_counts[word].most_common(1)[0][0]

# Example usage
corpus = [
    "the cat sat on the mat",
    "the dog sat on the rug",
    "the dog played with the cat",
    "the cat and the dog are friends"
]

model = MLELanguageModel()
model.train(corpus)

# Predict the next word for 'the'
print(model.predict_next_word('the'))

# Predict the next word for 'cat'
print(model.predict_next_word('cat'))


cat
sat


In [51]:
model = MLELanguageModel()
model.train(A_corpus)

res = []
word = 'Alice'
res.append(word)
for i in range(50):
    word = model.predict_next_word(word)
    res.append(word)
print(' '.join(res))

Alice was a little golden key, and the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project Gutenberg™ electronic works in the Project


In [57]:
model = MLELanguageModel()
model.train(A_corpus[500:800])

res = []
word = 'Alice'
res.append(word)
for i in range(50):
    word = model.predict_next_word(word)
    res.append(word)
print(' '.join(res))

Alice was a little timidly, "why your temper," said the same thing is a little timidly, "why your temper," said the same thing is a little timidly, "why your temper," said the same thing is a little timidly, "why your temper," said the same thing is a little timidly, "why your


In [61]:
#pip install sklearn-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 1.6 MB/s eta 0:00:001.5 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-crfsuite: filename=python_crfsuite-0.9.9-cp311-cp311-macosx_11_0_arm64.whl size=145221 sha256=a966ab3486e892c23a6f2c98cc433bc348e3e1369ef805d2d9d1596117f6b724
  Stored in directory: /Users/navid/Library/Caches/pip/wheels/4a/23/ab/586db2b4846c6de75693dec052c3cfc77e3c920f6a9ba97342
Successfully built python-crfsuite
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

class CRFLanguageModel:
    def __init__(self):
        """
        Initialize the CRF-based language model.
        """
        self.model = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True
        )
    
    def _create_features(self, sentence, index):
        """
        Create feature dict for a given word in a sentence.
        
        :param sentence: List of words in the sentence.
        :param index: Index of the word to create features for.
        :return: A dict of features.
        """
        word = sentence[index]
        features = {
            'word': word,
            'is_first': index == 0,
            'is_last': index == len(sentence) - 1,
            'prev_word': '' if index == 0 else sentence[index - 1],
            'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
            # More features can be added here
        }
        return features
    
    def _prepare_data(self, corpus):
        """
        Prepare training data for CRF, extracting features and labels.
        
        :param corpus: List of sentences to train on.
        :return: A tuple (X_train, y_train) for features and labels.
        """
        X_train = []
        y_train = []
        for sentence in corpus:
            words = sentence.split()
            X_sentence = [self._create_features(words, i) for i in range(len(words))]
            y_sentence = words[1:] + ['END']
            X_train.append(X_sentence)
            y_train.append(y_sentence)
        return X_train, y_train
    
    def train(self, corpus):
        """
        Train the CRF language model on the given corpus.
        
        :param corpus: List of sentences to train on.
        """
        X_train, y_train = self._prepare_data(corpus)
        self.model.fit(X_train, y_train)
    
    def generate(self, sequence, max_length=10):
        """
        Generate a continuation of the given word sequence using the trained model.
        
        :param sequence: The starting sequence of words.
        :param max_length: The maximum length of the continuation.
        :return: Generated sequence of words.
        """
        sentence = sequence.split()
        for _ in range(max_length):
            features = [self._create_features(sentence, len(sentence) - 1)]
            next_word = self.model.predict_single(features)[0]
            if next_word == 'END':
                break
            sentence.append(next_word)
        return ' '.join(sentence)

# Example usage:
corpus = [
    'the cat sat on the mat',
    'the dog played with the cat',
    'the quick brown fox jumps over the lazy dog',
    # ... more sentences
]

In [69]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

# Ensure that NLTK's tokenizers are available
nltk.download('punkt')

def read_book(file_path):
    """
    Read the book text from a file.
    
    :param file_path: Path to the book text file.
    :return: Raw text of the book.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def clean_text(text):
    """
    Perform initial text cleaning on the book text.
    
    :param text: Raw text of the book.
    :return: Cleaned text.
    """
    # Remove any Project Gutenberg-specific headers or footers, if present
    start_pattern = r"\*\*\* START OF (THE|THIS) PROJECT GUTENBERG EBOOK .+ \*\*\*"
    end_pattern = r"\*\*\* END OF (THE|THIS) PROJECT GUTENBERG EBOOK .+ \*\*\*"
    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)

    # If start and end patterns are found, extract the main content
    if start_match and end_match:
        text = text[start_match.end():end_match.start()]

    # Replace multiple whitespace with a single space
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def preprocess_for_crf(text):
    """
    Preprocess the cleaned text for CRF, including tokenization.
    
    :param text: Cleaned text of the book.
    :return: List of tokenized sentences.
    """
    # Split the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize each sentence into words
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

    return tokenized_sentences

# Path to your text file
file_path = 'Alice.txt'

# Read and preprocess the book text
raw_text = read_book(file_path)
cleaned_text = clean_text(raw_text)
tokenized_corpus = preprocess_for_crf(cleaned_text)

# Now tokenized_corpus can be used to create features for CRF
# Example of printing the first few preprocessed sentences
for sentence in tokenized_corpus[:5]:
    print(sentence)


['Produced', 'by', 'Jason', 'Isbell', ',', 'Irma', 'Spehar', ',', 'and', 'the', 'Online', 'Distributed', 'Proofreading', 'Team', 'at', 'http', ':', '//www.pgdp.net', '[', 'Illustration', ':', 'Alice', 'in', 'the', 'Room', 'of', 'the', 'Duchess', '.', ']']
['_THE', '``', 'STORYLAND', "''", 'SERIES_', 'ALICE', "'S", 'ADVENTURES', 'IN', 'WONDERLAND', 'SAM', "'", 'L', 'GABRIEL', 'SONS', '&', 'COMPANY', 'NEW', 'YORK', 'Copyright', ',', '1916', ',', 'by', 'SAM', "'", 'L', 'GABRIEL', 'SONS', '&', 'COMPANY', 'NEW', 'YORK', 'ALICE', "'S", 'ADVENTURES', 'IN', 'WONDERLAND', '[', 'Illustration', ']', 'I', '--', 'DOWN', 'THE', 'RABBIT-HOLE', 'Alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank', ',', 'and', 'of', 'having', 'nothing', 'to', 'do', '.']
['Once', 'or', 'twice', 'she', 'had', 'peeped', 'into', 'the', 'book', 'her', 'sister', 'was', 'reading', ',', 'but', 'it', 'had', 'no', 'pictures', 'or', 'conversations', 'in', 'it', ','

[nltk_data] Downloading package punkt to /Users/navid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:
data = cleaned_text.split('.')
data = [sentence.encode('utf-8') for sentence in data]

model = CRFLanguageModel()
model.train(data)

# Generate a sequence starting with 'the quick brown fox'
print(model.generate('Alice was'))

ValueError: The numbers of items and labels differ: |x| = 0, |y| = 1

In [91]:
from sklearn_crfsuite import CRF
import codecs



def create_features(sentence, index):
    """
    Create feature dict for a given word in a sentence.

    :param sentence: List of words in the sentence.
    :param index: Index of the word to create features for.
    :return: A dict of features.
    """
    word = sentence[index]
    features = {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        # More features can be added here
    }
    return features


crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)

X_train = []
y_train = []
for sentence in data:
    words = sentence.split()
    X_sentence = [create_features(words, i) for i in range(len(words))]
    y_sentence = words[1:] + ['END']
    X_train.append(X_sentence)
    y_train.append(y_sentence)

crf.fit(X_train, y_train)

ValueError: The numbers of items and labels differ: |x| = 0, |y| = 1

In [94]:
len(X_train)

404

In [95]:
len(y_train)

404